In [1]:
import os
os.chdir( "../")

In [2]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])


df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [3]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [4]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample



urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation, urm_test = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train_validation, train_percentage = 0.80)

In [6]:
num_users = len(userId_unique)
num_items = len(itemId_unique)

In [7]:
from Recommenders.Recommender_import_list import *
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10], ignore_users=[])

2023-12-28 14:05:54.100771: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-28 14:05:54.100800: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


EvaluatorHoldout: Ignoring 2602 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [8]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2128 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [9]:
import os

output_folder_path = "result_experiments_parallel/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 200  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [10]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt


In [11]:
earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [12]:
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = earlystopping_keywargs,     # Additional hyperparameters for the fit function
)

In [13]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = earlystopping_keywargs,     # Additional hyperparameters for the fit function
)

In [14]:
#hyperparameterSearch = SearchBayesianSkopt(recommender_class,
#                                         evaluator_validation=evaluator_validation,
#                                         evaluator_test=evaluator_test)

In [15]:
similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"]

In [16]:
from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative, runHyperparameterSearch_Content
from Recommenders.NonPersonalizedRecommender import TopPop, Random
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
import os, multiprocessing
from functools import partial


runHyperparameterSearch_Collaborative_partial = partial(runHyperparameterSearch_Collaborative,
       URM_train = urm_train,
       URM_train_last_test = urm_train_validation,
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)


In [17]:
from Recommenders.EASE_R.EASE_R_RecommenderV2 import EASE_R_RecommenderV2

collaborative_algorithm_list = [
    EASE_R_RecommenderV2
]

In [18]:
pool = multiprocessing.Pool(processes=int(multiprocessing.cpu_count()), maxtasksperchild=1)
pool.map(runHyperparameterSearch_Collaborative_partial, collaborative_algorithm_list)

SearchBayesianSkopt: Resuming 'EASE_R_RecommenderV2' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': None, 'normalize_matrix': False, 'l2_norm': 47124.07928361357}
EASE_R_RecommenderV2: URM Detected 472 ( 3.7%) users with no interactions.
EASE_R_RecommenderV2: URM Detected 341 ( 1.5%) items with no interactions.
EASE_R_RecommenderV2: Fitting model... 


/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


EASE_R_RecommenderV2: Fitting model... done in 1.84 min
Collected: 102
EvaluatorHoldout: Processed 10036 (100.0%) in 5.01 sec. Users per second: 2002
SearchBayesianSkopt: New best config found. Config 0: {'topK': None, 'normalize_matrix': False, 'l2_norm': 47124.07928361357} - results: PRECISION: 0.0372758, PRECISION_RECALL_MIN_DEN: 0.0830239, RECALL: 0.0761191, MAP: 0.0143485, MAP_MIN_DEN: 0.0331156, MRR: 0.1130630, NDCG: 0.0645018, F1: 0.0500446, HIT_RATE: 0.2921483, ARHR_ALL_HITS: 0.1271638, NOVELTY: 0.0039970, AVERAGE_POPULARITY: 0.6655867, DIVERSITY_MEAN_INTER_LIST: 0.4821204, DIVERSITY_HERFINDAHL: 0.9482072, COVERAGE_ITEM: 0.0709657, COVERAGE_ITEM_HIT: 0.0092251, ITEMS_IN_GT: 0.7413824, COVERAGE_USER: 0.7941130, COVERAGE_USER_HIT: 0.2319987, USERS_IN_GT: 0.7941130, DIVERSITY_GINI: 0.0026711, SHANNON_ENTROPY: 5.2654479, RATIO_DIVERSITY_HERFINDAHL: 0.9485507, RATIO_DIVERSITY_GINI: 0.0077393, RATIO_SHANNON_ENTROPY: 0.4059137, RATIO_AVERAGE_POPULARITY: 5.3907751, RATIO_NOVELTY: 0.282

/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


EASE_R_RecommenderV2: Fitting model... done in 1.79 min
Collected: 25
EvaluatorHoldout: Processed 10036 (100.0%) in 5.46 sec. Users per second: 1837
SearchBayesianSkopt: New best config found. Config 1: {'topK': None, 'normalize_matrix': False, 'l2_norm': 8.118686112989252} - results: PRECISION: 0.0512953, PRECISION_RECALL_MIN_DEN: 0.1052450, RECALL: 0.0953551, MAP: 0.0228472, MAP_MIN_DEN: 0.0484896, MRR: 0.1660782, NDCG: 0.0901720, F1: 0.0667065, HIT_RATE: 0.3667796, ARHR_ALL_HITS: 0.1941592, NOVELTY: 0.0046874, AVERAGE_POPULARITY: 0.3082374, DIVERSITY_MEAN_INTER_LIST: 0.9530801, DIVERSITY_HERFINDAHL: 0.9952985, COVERAGE_ITEM: 0.2239672, COVERAGE_ITEM_HIT: 0.0520655, ITEMS_IN_GT: 0.7413824, COVERAGE_USER: 0.7941130, COVERAGE_USER_HIT: 0.2912644, USERS_IN_GT: 0.7941130, DIVERSITY_GINI: 0.0303177, SHANNON_ENTROPY: 9.2112404, RATIO_DIVERSITY_HERFINDAHL: 0.9956590, RATIO_DIVERSITY_GINI: 0.0878425, RATIO_SHANNON_ENTROPY: 0.7100952, RATIO_AVERAGE_POPULARITY: 2.4965018, RATIO_NOVELTY: 0.3312

...

/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


EASE_R_RecommenderV2: Fitting model... done in 1.81 min
Collected: 202
EvaluatorHoldout: Processed 10036 (100.0%) in 5.42 sec. Users per second: 1853
SearchBayesianSkopt: Config 199 is suboptimal. Config: {'topK': None, 'normalize_matrix': False, 'l2_norm': 128.2271726426419} - results: PRECISION: 0.0592567, PRECISION_RECALL_MIN_DEN: 0.1224659, RECALL: 0.1108730, MAP: 0.0269429, MAP_MIN_DEN: 0.0563200, MRR: 0.1881772, NDCG: 0.1040086, F1: 0.0772348, HIT_RATE: 0.4090275, ARHR_ALL_HITS: 0.2243771, NOVELTY: 0.0045691, AVERAGE_POPULARITY: 0.3464759, DIVERSITY_MEAN_INTER_LIST: 0.9343001, DIVERSITY_HERFINDAHL: 0.9934207, COVERAGE_ITEM: 0.1481865, COVERAGE_ITEM_HIT: 0.0417604, ITEMS_IN_GT: 0.7413824, COVERAGE_USER: 0.7941130, COVERAGE_USER_HIT: 0.3248141, USERS_IN_GT: 0.7941130, DIVERSITY_GINI: 0.0185428, SHANNON_ENTROPY: 8.6551170, RATIO_DIVERSITY_HERFINDAHL: 0.9937806, RATIO_DIVERSITY_GINI: 0.0537259, RATIO_SHANNON_ENTROPY: 0.6672236, RATIO_AVERAGE_POPULARITY: 2.8062063, RATIO_NOVELTY: 0.32

/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


EASE_R_RecommenderV2: Fitting model... done in 1.81 min
Collected: 212
EvaluatorHoldout: Processed 10510 (100.0%) in 5.93 sec. Users per second: 1772
SearchBayesianSkopt: Best config evaluated with evaluator_test with constructor data for final test. Config: {'topK': None, 'normalize_matrix': False, 'l2_norm': 126.49349950908267} - results:
CUTOFF: 10 - PRECISION: 0.0840913, PRECISION_RECALL_MIN_DEN: 0.1510789, RECALL: 0.1292230, MAP: 0.0419725, MAP_MIN_DEN: 0.0752455, MRR: 0.2512038, NDCG: 0.1367664, F1: 0.1018828, HIT_RATE: 0.4957184, ARHR_ALL_HITS: 0.3216201, NOVELTY: 0.0046339, AVERAGE_POPULARITY: 0.3269265, DIVERSITY_MEAN_INTER_LIST: 0.9415967, DIVERSITY_HERFINDAHL: 0.9941507, COVERAGE_ITEM: 0.1526415, COVERAGE_ITEM_HIT: 0.0537305, ITEMS_IN_GT: 0.8014580, COVERAGE_USER: 0.8316189, COVERAGE_USER_HIT: 0.4122488, USERS_IN_GT: 0.8316189, DIVERSITY_GINI: 0.0209341, SHANNON_ENTROPY: 8.8549679, RATIO_DIVERSITY_HERFINDAHL: 0.9945097, RATIO_DIVERSITY_GINI: 0.0599839, RATIO_SHANNON_ENTROPY:

[None]